### This notebook opens stream video from your webcam (first webcam if more than 1 are detected) and gives prediction if you wearing the mask.
### Haarcascade face recogniser from open CV library is used to detect face in the camera and trained CNN model is used to make prediction
### Code for CNN model you will find in "mask_classifier.py" file
### To succesfully run this notebook you need to have "model_masks_trial2.h5" file in the same folder where you have this notebook and good illumination. The face recognizer classifier is not optimised for different illumination conditions and black masks (I am happy to hear idea why haarcascade face recongiser performs bad in my case with black masks).
### The main idea is to demonstrate the mask recognition. To stop video stream from your camera, press "Esc".

In [19]:
import cv2
import numpy as np
new_dim=(160,160)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') # importing haarcascade image classifier for face recognition
from keras.models import load_model
model = load_model('model_masks_trial2.h5', compile = True) # load trained model for mask detection

In [22]:
cam=cv2.VideoCapture(0) # connect to laptop webcam
cv2.namedWindow("Mask detection")
while True:
    ret, frame = cam.read() # read frame from camera video stream
    if not ret: # check if frame was taken
        print("failed to grab frame")
        break
    faces_detected = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) # applying haarcascade classifier to detect face on the frame
    if type(faces_detected)==tuple: # if face is not recognised show corresponding message on the screen
        clr=(255, 0, 255)
        txt="No face detected"
        cv2.putText(frame, txt, (int(frame.shape[0]/2), int(frame.shape[1]/2)), 1, 2, clr)
        cv2.imshow("Mask detection", frame)
    else:
        (x, y, w, h) = faces_detected[0] # get face coordinates
        face_croped=frame[y-20:y+h+20, x:x+w] # extract face
        try:
            X=cv2.resize(face_croped, new_dim, interpolation=cv2.INTER_LINEAR)
            v=X.shape
            X=np.reshape(X, [1, v[0], v[1], v[2]])
            y_pred=(model.predict(X) > 0.5).astype("int32") # apply "mask/no mask" prediction to frame
            prob=model.predict(X)[0][0]
            if y_pred==0:     
                prob=1-prob
                txt="No mask!"+str(round(prob*100, 3))+"%"
                clr=(0, 0, 255)
            else:
                txt="Mask!"+str(round(prob*100, 3))+"%"
                clr=(0, 255, 0)
            cv2.rectangle(frame, (x, y-20), (x+w, y+h+20), clr, 1)
            cv2.putText(frame, txt, (x,y), 1, 1, clr) # show message if mask or no mask is detected
            cv2.imshow("Mask detection", frame)
        except cv2.error as exp: # catch "bad" frames errors
            print("Invalid frame")
            
    k = cv2.waitKey(10)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
cam.release()

cv2.destroyAllWindows()

Invalid frame
Invalid frame
Invalid frame
Invalid frame
Invalid frame
Invalid frame
Invalid frame
Invalid frame
Invalid frame
failed to grab frame
